<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/SplitData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/WideTable/WideTable.xlsx')
df = df.iloc[:, 1:]
df.isnull().any()

PatientUID             False
PatientVisitUID        False
Slopes                 False
G1M2:ARB               False
G3M1:Statin            False
                       ...  
PricipalProblemCode     True
PricipalProblemName     True
InPatient               True
OutPatient              True
Emergency               True
Length: 83, dtype: bool

In [9]:
df

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,Sex,Age,NATNL,Location,PricipalProblemCode,PricipalProblemName,InPatient,OutPatient,Emergency,SlopeBins
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,หญิง,75.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0,1
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,หญิง,76.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0,4
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,หญิง,78.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0.0,1.0,0.0,1
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,หญิง,79.0,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,F411,Generalized anxiety disorder,0.0,1.0,0.0,0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,หญิง,58.0,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,NaN,NaN,0.0,1.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,ชาย,64.0,ไทย,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ,44054006,Type 2 diabetes mellitus,0.0,1.0,0.0,4
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,หญิง,72.0,ไทย,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ,44054006,Type 2 diabetes mellitus,0.0,1.0,0.0,1
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,ชาย,67.0,ไทย,TB (คลินิก)-D707,A150,"Tuberculosis of lung\, confirmed by sputum mic...",0.0,1.0,0.0,2
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,ชาย,68.0,ไทย,โรคเลือดและต่อมน้ำเหลือง (คลินิก)-D716,414166008,"Extranodal NK/T-cell lymphoma, nasal type",0.0,1.0,0.0,4


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Assuming you have your data loaded into a DataFrame called 'df'

# Create a new column 'SlopeBins' to bin the 'Slope' values
num_bins = 5  # You can adjust this number as needed
df['SlopeBins'] = pd.qcut(df['Slopes'], num_bins, labels=False)

# Initialize StratifiedKFold with 5 folds
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store train and test indices
train_indices = []
test_indices = []

# Loop through each fold
for train_index, test_index in skf.split(df, df['SlopeBins']):
    train_indices.append(train_index)
    test_indices.append(test_index)

# Create 5-fold cross-validation datasets
for fold in range(5):
    train_data = df.iloc[train_indices[fold]]
    test_data = df.iloc[test_indices[fold]]

    # Perform random selection for each class (each 'SlopeBin')
    sample_fraction = 0.1  # 10% of the total
    train_data_sampled = train_data.groupby('SlopeBins').apply(
        lambda group: group.sample(frac=sample_fraction, random_state=42))  # Add random_state for reproducibility

    # Reset the index of train_data_sampled
    train_data_sampled.reset_index(drop=True, inplace=True)

    # Reset the index of train_data
    train_data.reset_index(drop=True, inplace=True)

    # Separate the 10% samples as the test set
    test_data = pd.concat([test_data, train_data_sampled])
    train_data = train_data.drop(train_data_sampled.index)

    # Now, you have the train_data and test_data for the current fold
    # You can perform your model training and testing here
    # ...

# Remove the 'SlopeBins' column if you no longer need it
#df.drop('SlopeBins', axis=1, inplace=True)


In [14]:
print(train_data.shape)
print(test_data.shape)

(37886, 84)
(14733, 84)


In [15]:
bin_distribution = df['SlopeBins'].value_counts().sort_index()
bin_distribution

0    10524
1    10524
2    10523
3    10524
4    10524
Name: SlopeBins, dtype: int64

In [16]:
min_max_slopes = df.groupby('SlopeBins')['Slopes'].agg(['min', 'max'])
min_max_slopes.columns = ['Min Slope', 'Max Slope']
min_max_slopes

,Min Slope,Max Slope
SlopeBins,,
0,-79.002653,-7.834596
1,-7.832097,-3.322209
2,-3.322070,-0.246942
3,-0.246720,3.650000
4,3.657900,106.892857
